In [ ]:
pip install music21



In [ ]:
pip install tensorflow

In [ ]:
pip install keras

In [ ]:
pip install h5py

In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

from google.colab import drive
drive.mount('/content/drive')

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)


def get_notes():
    """ Get all the notes and chords from the midi files in the specified directory """
    notes = []
    # Make sure to use a wildcard pattern to match MIDI files
    midi_folder = "/content/drive/My Drive/Big Data Project/Romantic Midis copy 2/*.mid"

    for file in glob.glob(midi_folder):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            if s2:  # Check if the score has instrument parts
                notes_to_parse = s2.parts[0].recurse()
            else:  # Score doesn't have parts
                notes_to_parse = midi.flat.notes
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        ...
    import os

    notes_file_path = '/content/drive/My Drive/Big Data Project/data/notes'

    # Create 'data' directory in Google Drive if it does not exist
    notes_dir = os.path.dirname(notes_file_path)
    if not os.path.exists(notes_dir):
        os.makedirs(notes_dir)

    # Save the notes file to Google Drive
    with open(notes_file_path, 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 10

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # Correctly use to_categorical here
    network_output = to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """create the structure of the neural network"""
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "/content/drive/My Drive/Big Data Project/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )

    # Define the early stopping callback
    early_stopping = EarlyStopping(
        monitor='loss',
        patience=5,  # Number of epochs with no improvement after which training will be stopped
        verbose=1,
        mode='min'
    )

    callbacks_list = [checkpoint, early_stopping]

    model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)


if __name__ == '__main__':
    train_network()

Mounted at /content/drive
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Chopin, Frédéric, Mazurka in B-flat major 'Wołowska', B.73, eEnelX249O8.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Chopin, Frédéric, Valse mélancolique in F-sharp minor, A1_7, endJjZozRtM.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Chopin, Frédéric, Mazurkas, Op.50, h0BjbU9upFI.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Chopin, Frédéric, Piano Sonata No.1, Op.4, n-XhROuRDBY.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Debussy, Claude, La plus que lente, IazwSz8Ueh8.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Liszt, Franz, 2 Polonaises, S.223, VvreQKtsObY.mid
Parsing /content/drive/My Drive/Big Data Project/Romantic Midis copy 2/Chopin, Frédéric, Mazurkas, Op.24, flIQ02Nlkpk.mid
Parsing /content/drive/My Drive/Big Data Project/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4793/4794 [============================>.] - ETA: 0s - loss: 5.3374
Epoch 2: loss improved from 5.43927 to 5.33738, saving model to /content/drive/My Drive/Big Data Project/weights-improvement-02-5.3374-bigger.hdf5
4794/4794 [==============================] - 38s 8ms/step - loss: 5.3374
Epoch 3/200
4788/4794 [============================>.] - ETA: 0s - loss: 5.3304
Epoch 3: loss improved from 5.33738 to 5.33043, saving model to /content/drive/My Drive/Big Data Project/weights-improvement-03-5.3304-bigger.hdf5
4794/4794 [==============================] - 37s 8ms/step - loss: 5.3304
Epoch 4/200
4794/4794 [==============================] - ETA: 0s - loss: 5.3252
Epoch 4: loss improved from 5.33043 to 5.32522, saving model to /content/drive/My Drive/Big Data Project/weights-improvement-04-5.3252-bigger.hdf5
4794/4794 [==============================] - 37s 8ms/step - loss: 5.3252
Epoch 5/200
4794/4794 [==============================] - ETA: 0s - loss: 5.3217
Epoch 5: loss improved from 5.32

In [ ]:
!ls /content



sample_data


In [1]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation


from google.colab import drive
drive.mount('/content/drive')


def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('/content/drive/My Drive/Big Data Project/data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 10
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """create the structure of the neural network"""

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))  # Ensure n_vocab matches the training phase
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    model.load_weights('/content/drive/My Drive/Big Data Project/weights-improvement-19-5.2630-bigger.hdf5')

    return model


def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    # Specify the full path including the folder name where you want to save the file in Google Drive
    file_path = '/content/drive/My Drive/Big Data Project/test_output.mid'
    midi_stream.write('midi', fp=file_path)


if __name__ == '__main__':
    generate()


Mounted at /content/drive
